In [195]:
from model.model import *
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [192]:
df = pd.read_csv("data/ml-1m/movielens.test.negative")

In [197]:
df.head()

,movieID,rating,timestamp,userID
0,2485,0,NaN,3982
1,541,1,960935593.0,5307
2,1699,0,NaN,204
3,3079,1,974654075.0,2264
4,3687,0,NaN,2553


In [44]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32,label="rating"):
  dataframe = dataframe.copy()
  _ = dataframe.pop("timestamp")
  labels = dataframe.pop('rating')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [45]:
train_ds = df_to_dataset(df, batch_size=2)
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('Movieid:', feature_batch['movieID'])
  print('A batch of targets:', label_batch )

Every feature: ['movieID', 'userID']
Movieid: tf.Tensor([589 617], shape=(2,), dtype=int32)
A batch of targets: tf.Tensor([0 0], shape=(2,), dtype=int32)


## Example - raw

In [ ]:
user = tf.ragged.constant([["hej","hej","hej"]]
                         ) # Input movies, string in this case 

item = tf.ragged.constant([["hej"]]) 

# Create an embedding table.
num_buckets = 10 # number of movies....
embedding_size = 4 # embedding size

ans = tf.keras.initializers.TruncatedNormal() # Initialization the embeddings 
embedding_table = tf.Variable(ans([num_buckets, embedding_size])) # Embedding table 

# Look up the embedding for each word.
user_buckets = tf.strings.to_hash_bucket_fast(user, num_buckets)
user_embeddings = tf.ragged.map_flat_values(
    tf.nn.embedding_lookup, embedding_table, user_buckets)
item_bucket = tf.strings.to_hash_bucket_fast(item, num_buckets)
item_embedding = tf.ragged.map_flat_values(
    tf.nn.embedding_lookup, embedding_table, item_bucket) 

## Go Go Go class

In [246]:
class GetEmbeddings(tf.keras.layers.Layer):

    def __init__(self, num_buckets=None, embedding_size=None):
        super(GetEmbeddings, self).__init__()
        self.num_buckets = num_buckets
        self.embedding_size = embedding_size
    
    def build(self, input_shape):
        pass
    
    def call(self, item, user):
        """Lets do it here ........"""
        initiator = tf.keras.initializers.TruncatedNormal() # Initialization the embeddings 
        embedding_table = tf.Variable(initiator([self.num_buckets, self.embedding_size])) # Embedding table 
        # Look up the embedding for each word.
        user_buckets = tf.strings.to_hash_bucket_fast(user, self.num_buckets)
        user_embeddings = tf.ragged.map_flat_values(
            tf.nn.embedding_lookup, embedding_table, user_buckets)
        item_bucket = tf.strings.to_hash_bucket_fast(item, self.num_buckets)
        item_embedding = tf.ragged.map_flat_values(
            tf.nn.embedding_lookup, embedding_table, item_bucket)
        output = tf.math.multiply(item_embedding[0],user_embeddings)
        output = tf.reduce_mean(out,axis=0)
        return output

In [247]:
layer = GetEmbeddings(10,4)
user = tf.ragged.constant([["hej","hej","hej"]]) # Input movies, string in this case 
item = tf.ragged.constant([["hej"]]) 
layer(item,user)


<tf.Tensor: id=24841, shape=(4,), dtype=float32, numpy=
array([8.3136437e-03, 2.0645888e-05, 2.8431609e-03, 4.3349504e-03],
      dtype=float32)>